Transformar coluna de store segment em 2

# Funções

In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from math import ceil  # Importar a função ceil

from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid, KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

pd.options.display.max_columns=70 
pd.options.display.max_rows=80

In [2]:
def find_missing_percent(data):
    """
    Retorna dataframe contendo o total de valores faltantes e porcentagem do total
    de valores faltantes da coluna.
    """
    miss_df = pd.DataFrame({'ColumnName':[],'TotalMissingVals':[],'PercentMissing':[]})
    for col in data.columns:
        sum_miss_val = data[col].isnull().sum()
        percent_miss_val = round((sum_miss_val/data.shape[0])*100,2)
        miss_df.loc[len(miss_df)] = dict(zip(miss_df.columns,[col,sum_miss_val,percent_miss_val]))
    return miss_df

In [3]:
def find_correlated_columns(df, interval):
    """
    Encontra e exibe as correlações entre colunas de um DataFrame.

    Parâmetros:
    - df: DataFrame pandas
    - intervalo de correlação desejado (uma tupla de dois valores)

    Retorna:
    - Lista de tuplas representando pares de colunas correlacionadas.
    """
    correlation_matrix = df.corr(numeric_only=True)
    correlated_columns = []

    # Iterar sobre as combinações de colunas para encontrar correlações
    for i in range(len(correlation_matrix.columns)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            corr = correlation_matrix.iloc[i, j]
            if interval[0] <= abs(corr) <= interval[1]:
                col1 = correlation_matrix.columns[i]
                col2 = correlation_matrix.columns[j]
                correlated_columns.append((col1, col2))
                print(f"Correlação entre {col1} e {col2}: {corr}")

    # Plotar um mapa de calor da matriz de correlação
    plt.figure(figsize=(20, 16))
    sns.heatmap(correlation_matrix, annot=True, cmap='cubehelix_r')
    plt.title('Matriz de Correlação')
    plt.xlabel('Variáveis')
    plt.ylabel('Variáveis')
    plt.show()

    return correlated_columns

In [4]:
def correlacao_com_variavel_alvo(df, target_variable, nivel="forte", top_n=5):
    """
    Imprime as n features com as maiores correlações com uma variável alvo, com base no nível escolhido.

    Parâmetros:
    - df: DataFrame pandas.
    - target_variable: String, nome da variável alvo.
    - nivel: String que define o critério de correlação ("forte", "fraca", etc.).
    - top_n: Número inteiro, quantidade de features a serem impressas.

    Retorna:
    - Nenhum (imprime as correlações).
    """
    correlation_matrix = df.corr(numeric_only=True)

    # Filtra as correlações com base no nível escolhido
    if nivel.lower() == "forte":
        filtered_correlations = correlation_matrix[((correlation_matrix >= 0.7) & (correlation_matrix < 1.0)) | ((correlation_matrix <= -0.7) & (correlation_matrix > -1.0))]
    else:
        raise ValueError("Nível não suportado. Atualmente, apenas 'forte' é suportado.")

    # Filtra as correlações com a variável alvo
    correlations_with_target = filtered_correlations[target_variable].sort_values(ascending=False)

    # Pegar as n maiores correlações
    top_n_correlations = correlations_with_target.head(top_n)

    # Imprimir as n maiores correlações com a variável alvo
    print(f"As {top_n} maiores correlações com '{target_variable}' ({nivel}):")
    for feature, correlation in top_n_correlations.items():
        print(f"{feature}: {correlation}")


# Tratamento de Dados

Os valores outliers já foram tratados na etapa de limpeza dos dados, agora cabe lidar com os valores nulos para viabilizar a utilização dos modelos de Machine Learning

In [5]:
df1 = pd.read_csv('base_limpa1.csv', index_col=0)
df2 = pd.read_csv('base_limpa2.csv', index_col=0)

df = pd.concat([df1, df2])

In [6]:
len(df[~df['order_status'].eq('CANCELED')])

336425

In [7]:
# Mantendo somente os pedidos que não foram cancelados para que possamos analisar os tempos de entrega
# São pouco mais de 30k em 350k de linhas disponíveis
df = df[~df['order_status'].eq('CANCELED')]

df.reset_index(drop=True, inplace=True)

# removendo colunas que não serão úteis para prever o tempo de entrega, já que não teremos informações delas no momento do pedido
col_del = ['order_status', 'order_moment_ready', 'order_moment_collected',
       'order_moment_in_expedition', 'order_moment_delivering',
       'order_moment_finished', 'order_metric_collected_time',
       'order_metric_paused_time', 'order_metric_production_time',
       'order_metric_walking_time', 'order_metric_expediton_speed_time',
       'order_metric_transit_time']
df.drop(col_del, axis=1, inplace=True)


In [8]:
find_missing_percent(df)

,ColumnName,TotalMissingVals,PercentMissing
0,hub_name,0,0.00
1,hub_city,0,0.00
2,hub_state,0,0.00
3,store_name,0,0.00
4,store_segment,0,0.00
5,store_plan_price,77730,23.10
6,driver_modal,6118,1.82
7,driver_type,6118,1.82
8,delivery_distance_meters,2712,0.81
9,delivery_status,2681,0.80


In [9]:
# Removendo pedidos sem o tempo de entrega 'order_metric_cycle_time'
df = df[~df['order_metric_cycle_time'].isnull()]

In [10]:
#df['store_plan_price'].fillna(0, inplace=True)

In [11]:
#df['order_status'].value_counts()

In [12]:
#df['payment_status'].isnull()]

In [13]:
'''
for column in ['driver_modal', 'driver_type', 'payment_method', 'payment_status']:
    df[column].fillna(df[column].mode()[0], inplace=True)

for column in ['delivery_distance_meters',  'payment_amount', 'payment_fee', 
                'order_delivery_cost']:
    df[column].fillna(df[column].median(), inplace=True)



# Preencher NaN em 'delivery_status' com 'DELIVERED' onde 'order_metric_cycle_time' não é NaN
df.loc[df['order_metric_cycle_time'].notnull() & df['delivery_status'].isnull(), 'delivery_status'] = 'DELIVERED'

# Calcular o segundo valor mais frequente em 'delivery_status'
second_most_frequent_value = df['delivery_status'].value_counts().index[1]

# Preencher NaN em 'delivery_status' com o segundo valor mais frequente
df['delivery_status'].fillna(second_most_frequent_value, inplace=True)


# Condição para preencher apenas quando delivery_status for 'DELIVERED'
condition = df['delivery_status'] == 'DELIVERED'

# Lista das colunas a serem preenchidas com a mediana
columns_to_fill = [
    'order_metric_production_time', 
    'order_metric_expediton_speed_time', 
    'order_metric_transit_time', 
    'order_metric_collected_time', 
    'order_metric_paused_time', 
    'order_metric_walking_time',
    'order_metric_cycle_time'  # Adicionando 'order_metric_cycle_time' à lista
]

# Preencher os valores faltantes com a mediana nas colunas específicas quando delivery_status for 'DELIVERED'
for column in columns_to_fill:
    median_value = df.loc[condition, column].median()
    df.loc[condition & df[column].isnull(), column] = median_value
'''

"\nfor column in ['driver_modal', 'driver_type', 'payment_method', 'payment_status']:\n    df[column].fillna(df[column].mode()[0], inplace=True)\n\nfor column in ['delivery_distance_meters',  'payment_amount', 'payment_fee', \n                'order_delivery_cost']:\n    df[column].fillna(df[column].median(), inplace=True)\n\n\n\n# Preencher NaN em 'delivery_status' com 'DELIVERED' onde 'order_metric_cycle_time' não é NaN\ndf.loc[df['order_metric_cycle_time'].notnull() & df['delivery_status'].isnull(), 'delivery_status'] = 'DELIVERED'\n\n# Calcular o segundo valor mais frequente em 'delivery_status'\nsecond_most_frequent_value = df['delivery_status'].value_counts().index[1]\n\n# Preencher NaN em 'delivery_status' com o segundo valor mais frequente\ndf['delivery_status'].fillna(second_most_frequent_value, inplace=True)\n\n\n# Condição para preencher apenas quando delivery_status for 'DELIVERED'\ncondition = df['delivery_status'] == 'DELIVERED'\n\n# Lista das colunas a serem preenchidas 

# Modelagem

In [14]:
# Vamos listar as features que vamos utilizar
input_cols_categoric = ['hub_state', 'store_segment', 'channel_type'] #ADICIONAR 'driver_modal' DPS
input_cols_numeric = ['delivery_distance_meters', 'payment_amount', 'order_amount', 'order_moment_accepted', 'weekday']
features = input_cols_categoric + input_cols_categoric

target = 'order_metric_cycle_time'

In [15]:
# Dividindo os dados antes de qualquer pré-processamento
X = df[features]  # Features
y = df[target]  # Labels

# Divide em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=999
)

In [16]:
# Tratando as variáveis categóricas 

encoder = ce.OneHotEncoder(cols=input_cols_categoric)

X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)

AttributeError: 'DataFrame' object has no attribute 'unique'

In [ ]:
clf_RF = RandomForestRegressor()  # instanciar (= inicializar, criar o objeto)
clf_RF.fit(X_train, y_train)  # treina o modelo

ValueError: could not convert string to float: 'SP'

In [ ]:
# aplica no teste
y_pred = clf_RF.predict(X_test)

y_pred